In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [21]:
device = torch.device("cuda")

In [22]:
tokenizer_boltalka = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')
model_boltalka = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2').to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
def respond_to_dialog(model, texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer_boltalka(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer_boltalka.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=10,
        length_penalty=0.01,
        pad_token_id=tokenizer_boltalka.eos_token_id
    )
    decoded = tokenizer_boltalka.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [24]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(model_boltalka, history[-10:])
    next_sentence = input(result + '\n')
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
прилавок
кухня
пелмень


KeyboardInterrupt: Interrupted by user

In [15]:
history = ["Привет"]

In [19]:
result = respond_to_dialog(history[-10:])
next_sentence = input(result + '\n')
history.append(result)
history.append(next_sentence)

Нет.

